In [1]:
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [ ]:
def train(kernels, samples, chans, X, Y, file, epochs):
    Accuracies = []
    p_acc = []
    s_acc = []
    for index_1 in range(0, 7):
        for index_2 in range(7, 14):
            # build training set from allfiles expect the ones with index_1 and index_2
            X_train = [X[i] for i in range(len(X)) if i != index_1 and i != index_2]
            Y_train = [Y[i] for i in range(len(Y)) if i != index_1 and i != index_2]

            # build test set from the files with index_1 and index_2
            X_test = [X[i] for i in range(len(X)) if i == index_1 or i == index_2]
            Y_test = [Y[i] for i in range(len(Y)) if i == index_1 or i == index_2]

            X_train = np.concatenate(X_train)
            Y_train = np.concatenate(Y_train)

            # convert data to NHWC (trials, channels, samples, kernels) format.
            X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)

            model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

            # compile the model and set the optimizers
            model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

            # count number of parameters in the model
            numParams    = model.count_params()     

            # set a valid path for the system to record model checkpoints
            # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

            class_weights = {0:1, 1:1}

            fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = epochs, verbose = 2, 
                                    class_weight=class_weights)
            
            #open file to write results in
            f = open(file, "a") 

            # make prediction on the 2 test files individually
            X_test_1 = X_test[0].reshape(X_test[0].shape[0], chans, samples, kernels)
            X_test_2 = X_test[1].reshape(X_test[1].shape[0], chans, samples, kernels)

            probs_1 = model.predict(X_test_1)
            probs_2 = model.predict(X_test_2)
            preds_1 = probs_1.argmax(axis = -1)
            preds_2 = probs_2.argmax(axis = -1)
            acc_1 = balanced_accuracy_score(Y_test[0].argmax(axis=-1), preds_1)
            acc_2 = balanced_accuracy_score(Y_test[1].argmax(axis=-1), preds_2)
            f.write("----------------------SELECTED FILES: %d and %d---------------------\n" % (index_1, index_2))
            f.write("Classification accuracy for P: %f \n" % (acc_1))
            f.write("Classification accuracy for S: %f \n" % (acc_2))

            p_acc.append(acc_1)
            s_acc.append(acc_2)

            # make prediction on the 2 test files together
            X_test = np.concatenate(X_test)
            X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)
            Y_test = np.concatenate(Y_test)


            probs       = model.predict(X_test)
            preds       = probs.argmax(axis = -1)
            acc         = balanced_accuracy_score(Y_test.argmax(axis=-1), preds)
            f.write("Classification accuracy for P and S: %f \n" % (acc))
            f.write("\n")

            f.close()

            Accuracies.append(acc)

    return Accuracies, p_acc, s_acc

In [ ]:
def print_results(p_acc, s_acc, Accuracies, file):
    # calculate the mean and std of the accuracies
    p_acc = np.array(p_acc)
    s_acc = np.array(s_acc)
    acc = np.array(Accuracies)

    f = open(file, "a")
    f.write("Mean overall accuracy: %f \n" % (np.mean(acc)))
    f.write("Std overall accuracy: %f \n" % (np.std(acc)))
    f.write("Mean P accuracy: %f \n" % (np.mean(p_acc)))
    f.write("Std P accuracy: %f \n" % (np.std(p_acc)))
    f.write("Mean S accuracy: %f \n" % (np.mean(s_acc)))
    f.write("Std S accuracy: %f \n" % (np.std(s_acc)))
    f.close()
